In [1]:
import pandas as pd

data = pd.read_csv("github_issues.csv")

KeyboardInterrupt: 

In [4]:
data['body']

0          can't load the addon. issue to: https://github...
1          user experience: user who depends on screen re...
2          ┆attachments: <a href= https:& x2f;& x2f;githu...
3          gitlo = github x trello\n---\nthis board is no...
4          ┆attachments: <a href= https:& x2f;& x2f;githu...
                                 ...                        
5332148    методы класса: - instances, который возвращает...
5332149    - bug exists release version 1.1.0 master bran...
5332150    java.lang.linkageerror: loader constraint viol...
5332151    view details in rollbar: https://rollbar.com/b...
5332152    view details in rollbar: https://rollbar.com/j...
Name: body, Length: 5332153, dtype: object